In [1]:
import numpy as np
import pandas as pd
from collections import Counter

In [2]:
stopwords = pd.read_csv('./assets/data/stop_words.csv')
sw = stopwords['a'].tolist()
sw.append('a')

In [3]:
#creat a df with only Date and abstract
df = pd.read_excel('./demo_dataset.xlsx')
dframe = df[['Date','Abstract']]
dframe = dframe.dropna()
dframe['Abstract'] = dframe['Abstract'].apply(lambda x: ''.join([i.lower() for i in x if i.isalpha() or i == " "]) if x != None else x)\
                                      .apply(lambda x: ' '.join([i for i in x.split() if i not in sw]) if x != None else x)\
                                      .apply(lambda x: ' '.join(list(set(x.split()))) if x != None else x)
dframe.head()

,Date,Abstract
0,2020-06-20,minutes representations quantization lowering ...
1,2020-05-16,convolutionaugmented transformer competitive n...
2,2020-10-12,networks single indicates models v counterpart...
3,2020-06-08,limit variations use problem ease specifically...
4,2020-05-13,normal air confined speech dehydration per cau...


In [4]:
#for each row, append to a string str
str = ''
for index, row in dframe.iterrows():
    str = str + row['Abstract']

In [5]:
#counter the words
c = Counter(str.split())
df3 = pd.DataFrame.from_dict(c, orient='index').reset_index()
df3.columns = ['word', 'count']
df3 = df3.sort_values(by=['count'], ascending=False)
df3 = df3.head(500)
df3 = df3.reset_index(drop=True)
df3.head(10)

,word,count
0,speech,15740
1,results,7748
2,using,6580
3,study,5821
4,data,5540
5,model,5364
6,paper,5245
7,used,4968
8,also,4889
9,performance,4887


In [6]:
#save df3 to csv
df3.to_csv('./assets/data/mostCommonWords.csv', index=False)

### 手动筛选

In [7]:
#read from wordCount.csv
df3 = pd.read_csv('./assets/mostCommonWords.csv')
#make a dictionary with key as words and value as 0
d = dict.fromkeys(df3['word'].tolist(), 0)
d

FileNotFoundError: [Errno 2] No such file or directory: 'mostCommonWords.csv'

In [ ]:
#group by date
df3 = dframe.groupby('Date')
df3 = df3.apply(lambda x: x.sort_values(['Date'], ascending=[True]))
#delete the "Date" column
df3 = df3.reset_index(drop=True)
df3.head(10)

,Date,Abstract
0,2020-01-01,standardized ameliorate contributions signals ...
1,2020-01-01,childreported 鈭 invariance scores middle curre...
2,2020-01-01,tutdcase analysis model based learning effecti...
3,2020-01-01,converters article voice quality detector show...
4,2020-01-01,albeit use hallucinatory model based inconsist...
5,2020-01-01,video quest ryerson facial since human tested ...
6,2020-01-01,normalising processing invariance speaker mode...
7,2020-01-01,investigations improvement body scores exists ...
8,2020-01-01,diversity based taskbased diverse memory relat...
9,2020-01-01,encouraged processing usable lowfrequency majo...


In [ ]:
#connect all abstracts in the same date to one string
df4 = df3.groupby('Date')['Abstract'].apply(lambda x: "%s" % ' '.join(x))
df4 = pd.DataFrame(df4)
#reset index
df4 = df4.reset_index()
df4.head(10)

,Date,Abstract
0,2020-01-01,standardized ameliorate contributions signals ...
1,2020-01-02,bringing focused processing dl speaker learnin...
2,2020-01-03,exoskeletons bracelets technology direction si...
3,2020-01-04,review processing learning memory motorspeech ...
4,2020-01-05,definitive model confirm within outcomes rehab...
5,2020-01-06,rate use fusion establishes addressed error in...
6,2020-01-07,rate model trajectories outcomes show nsmi dat...
7,2020-01-08,stateoftheart must best since significantly ti...
8,2020-01-09,activity peaking shows videoclips marker alone...
9,2020-01-10,shows people adversarial intelligibility voice...


In [ ]:
#new df to create barcharrace
df5 = pd.DataFrame(columns=['Date', 'Word', 'Count'])

#for each abstract, call remove function to remove stopwords and punctuations
for index, row in df4.iterrows():
    #for each word in the abstract, add 1 to the dictionary
    for word in row['Abstract'].split():
        #if wor in the dictionary, add 1
        if word in d:
            d[word] += 1
    #sort the dictionary by value
    d = {k: v for k, v in sorted(d.items(), key=lambda item: item[1], reverse=True)}
    #get the first 8 words and save to the dataframe use concat
    for i in range(8):
        df5 = pd.concat([df5, pd.DataFrame([[row['Date'], list(d.keys())[i], list(d.values())[i]]], columns=['Date', 'Word', 'Count'])])
df5.head(10)

,Date,Word,Count
0,2020-01-01,speech,103
0,2020-01-01,results,56
0,2020-01-01,using,45
0,2020-01-01,study,43
0,2020-01-01,data,37
0,2020-01-01,used,32
0,2020-01-01,also,31
0,2020-01-01,paper,30
0,2020-01-02,speech,134
0,2020-01-02,results,74


In [ ]:
#save to csv
df5.to_csv('./assets/data/barChartRace.csv', index=False)